# CatBoost tutorials Обучающие программы CatBoost

# 1 Data Preparation Подготовка данных

## 1.1 CatBoost installation

Если Вы уже не установили CatBoost, можно сделать так путем выполнения'! победите устанавливают catboost' команда. Также необходимо установить ipywidgets пакет и выполнить специальную команду прежде, чем запустить jupyter ноутбук, чтобы построить графики.

In [1]:
!pip install catboost
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

!pip install sklearn

     |████████████████████████████████| 67.4 MB 24 kB/s 
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


**1.2 Data Loading Загрузка данных**

In [2]:
from catboost.datasets import titanic
import numpy as np


In [3]:
train_df, test_df = titanic()
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 1.3 Analysis of data Анализ данных

В первую очередь, давайте проверим, сколько делают отсутствующих ценностей мы имеем:

In [4]:
null_value_stats = train_df.isnull().sum(axis=0)
null_value_stats[null_value_stats != 0]

Age         177
Cabin       687
Embarked      2
dtype: int64

Как мы  видим, Age , Cabin и Embarked    действительно имеют некоторые недостающие ценности, поэтому давайте заполним их некоторым числом, исходя из их распределений - таким образом, модель была бы в состоянии легко различать их и принять его во внимание:

In [5]:
train_df.fillna(-999, inplace=True)
test_df.fillna(-999, inplace=True)

Теперь давайте отделим особенности и маркируем целевую переменную:

In [6]:
X = train_df.drop('Survived', axis=1) # axis - ось. Для дополнительной ясности можно выбрать axis='index' (вместо axis=0) или axis='columns' (вместо axis=1).
y = train_df.Survived

Обратите внимание, что наши особенности имеют differnt типы -  некоторые из них числовые, некоторые категорийные, и некоторые - даже просто последовательности, которые обычно должны обрабатываться в некотором особенном методе (например, закодированный с представлением сумки слов). Но в нашем случае мы могли рассматривать эти особенности последовательности так же, как категоричные одна - весь тяжелый подъем сделан в CatBoost. Насколько прохладный это?:)

In [7]:
print(X.dtypes)

categorical_features_indices = np.where(X.dtypes != float)[0]

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object


np.float устарело. Нужно просто float

# 1.4 Data Splitting Разделение данных

Давайте разделим данные на наборы обучения и проверки.

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.75, random_state=42)

X_test = test_df

# 2. Catboost Basics Основы Catboost

Цель
Выполните перекрестную проверку набора данных.

Набор данных разбивается на N складок. N–1 складки используются для обучения, а один склад используется для оценки производительности модели. N моделей обновляются на каждой итерации K. Каждая модель оценивается по собственному набору данных проверки на каждой итерации. Это приводит к N метрическим значениям на каждой итерации K.

Функция вычисляет среднее значение этих значений N и стандартное отклонение. Таким образом, эти два значения возвращаются на каждой итерации.cv

Если набор данных содержит идентификаторы групп, все объекты из одной группы добавляются в одну и ту же складку при выполнении секционирования.

Давайте сделаем необходимый импорт.

In [9]:
from catboost import CatBoostClassifier, Pool, cv
from sklearn.metrics import accuracy_score

# 2.1 Model Training Обучение модели

Теперь давайте создадим саму модель: Мы пошли бы сюда с параметрами по умолчанию (поскольку они обеспечивают действительно хорошее основание почти все время), единственной вещью, которую Мы хотели бы определить здесь, является custom_loss параметр, поскольку это дало бы нам способность видеть то, что продолжается с точки зрения этой метрики соревнования - точность, а также быть в состоянии наблюдать за logloss, поскольку это было бы более гладко на наборе данных такого размера.

In [10]:
model = CatBoostClassifier(
    custom_loss=['Accuracy'], # метрика потерь
    random_seed=42,           # случайное число, используемое для обчения
    logging_level='Silent' # Регистрируемый уровень для вывода выодных данных. Варианты:
                           # Silent (не производить информацию)
                           # Verbose (подробная информация)
                           # Info (дополнительная инфоромация)
                           # Debug (отладочная инфоормация)
)

In [11]:
model.fit(
    X_train, y_train,
    cat_features=categorical_features_indices, # Одномерный массив категориальных индексов столбцов.
    eval_set=(X_validation, y_validation),
#     logging_level='Verbose',  # можно не прокомментировать это для текстового вывода
    plot=True # тип метрики визуализации
);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Как Вы видите, возможно наблюдать, что наша модель учится через многословную продукцию или с хорошими сюжетами (лично, я определенно пошел бы со вторым вариантом - просто проверяют те сюжеты: Вы можете, например, увеличить сферы интересов!)

С этим мы видим, что лучшая ценность точности 0,8340 (на наборе проверки) была достигнута на 157 повышающих шагах.

# 2.2 Model Cross-Validation  Перекрестная проверка модели

Хочется утвердить Вашу модель, но после перекрестной проверки утвердить ее - еще лучше. И также с сюжетами! Таким образом без большего количества слов:

In [ ]:
cv_params = model.get_params()
cv_params.update({
    'loss_function': 'Logloss'
})
cv_data = cv(
    Pool(X, y, cat_features=categorical_features_indices),
    cv_params,
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Теперь у нас есть ценности наших функций потерь в каждом повышающем шаге, усредненном 3 сгибами, которые должны предоставить нам более точную оценку нашего образцового выступления:

In [ ]:
print('Best validation accuracy score: {:.2f}±{:.2f} on step {}'.format(
    np.max(cv_data['test-Accuracy-mean']),
    cv_data['test-Accuracy-std'][np.argmax(cv_data['test-Accuracy-mean'])],
    np.argmax(cv_data['test-Accuracy-mean'])
))

Best validation accuracy score: 0.83±0.02 on step 543


In [ ]:
print('Precise validation accuracy score: {}'.format(np.max(cv_data['test-Accuracy-mean'])))

Precise validation accuracy score: 0.8260381593714928


Как мы видим, наша начальная оценка работы на единственном сгибе проверки была слишком оптимистична - именно поэтому, перекрестная проверка так важна!

# 2.3 Model Applying  Применение модели

Все, что Вы должны сделать, чтобы получить прогнозы,

In [ ]:
predictions = model.predict(X_test)
predictions_probs = model.predict_proba(X_test)
print(predictions[:10])
print(predictions_probs[:10])

[0 0 0 0 1 0 1 0 1 0]
[[0.85473931 0.14526069]
 [0.76313031 0.23686969]
 [0.88972889 0.11027111]
 [0.87876173 0.12123827]
 [0.3611047  0.6388953 ]
 [0.90513381 0.09486619]
 [0.33434185 0.66565815]
 [0.78468564 0.21531436]
 [0.39429048 0.60570952]
 [0.94047549 0.05952451]]


Но давайте попытаемся добраться, лучшие прогнозы и особенности Catboost помогают нам в нем.

# 3 CatBoost Features CatBoost особенности 

Вы, возможно, заметили, что на образцовом создании ступают, я определил не только custom_loss, но также и random_seed параметр. Это было сделано, чтобы сделать этот ноутбук восстанавливаемым - по умолчанию catboost, выбирает некоторую случайную стоимость для семени:

In [ ]:
model_without_seed = CatBoostClassifier(iterations=10, logging_level='Silent')
model_without_seed.fit(X, y, cat_features=categorical_features_indices)

print('Random seed assigned for this model: {}'.format(model_without_seed.random_seed_))

Random seed assigned for this model: 0


Давайте определим некоторый params и давайте создадим Бассейн для большего удобства. Это хранит всю информацию о наборе данных (особенности, этикетки, категорические индексы особенностей, веса и и намного больше).

In [ ]:
params = {
    'iterations': 500,
    'learning_rate': 0.1,
    'eval_metric': 'Accuracy',
    'random_seed': 42,
    'logging_level': 'Silent',
    'use_best_model': False
}
train_pool = Pool(X_train, y_train, cat_features=categorical_features_indices)
validate_pool = Pool(X_validation, y_validation, cat_features=categorical_features_indices)

# 3.1 Using the best model Использование лучшей модели

Если Вам по существу установили проверку, то всегда лучше использовать use_best_model параметр во время обучения. По умолчанию этот параметр позволен. Если это позволено, получающийся ансамбль деревьев сжимается к лучшему повторению.

In [ ]:
model = CatBoostClassifier(**params)
model.fit(train_pool, eval_set=validate_pool)

best_model_params = params.copy()
best_model_params.update({
    'use_best_model': True
})
best_model = CatBoostClassifier(**best_model_params)
best_model.fit(train_pool, eval_set=validate_pool);

print('Simple model validation accuracy: {:.4}'.format(
    accuracy_score(y_validation, model.predict(X_validation))
))
print('')

print('Best model validation accuracy: {:.4}'.format(
    accuracy_score(y_validation, best_model.predict(X_validation))
))

Simple model validation accuracy: 0.8027

Best model validation accuracy: 0.8251


# 3.2 Early Stopping Ранняя остановка

Если Вам по существу установили проверку, то всегда легче и лучше использовать раннюю остановку. Эта особенность подобна предыдущей, но только в дополнение к улучшению качества это все еще экономит время.

In [ ]:
%%time
model = CatBoostClassifier(**params)
model.fit(train_pool, eval_set=validate_pool)

CPU times: user 4.96 s, sys: 343 ms, total: 5.31 s
Wall time: 2.89 s


In [ ]:
%%time
earlystop_params = params.copy()
earlystop_params.update({
    'od_type': 'Iter',
    'od_wait': 40
})
earlystop_model = CatBoostClassifier(**earlystop_params)
earlystop_model.fit(train_pool, eval_set=validate_pool);

CPU times: user 719 ms, sys: 54.8 ms, total: 774 ms
Wall time: 453 ms


In [ ]:
print('Simple model tree count: {}'.format(model.tree_count_))
print('Simple model validation accuracy: {:.4}'.format(
    accuracy_score(y_validation, model.predict(X_validation))
))
print('')

print('Early-stopped model tree count: {}'.format(earlystop_model.tree_count_))
print('Early-stopped model validation accuracy: {:.4}'.format(
    accuracy_score(y_validation, earlystop_model.predict(X_validation))
))

Таким образом, мы получаем лучшее качество в более короткое время.

Хотя, как был показан, более ранняя простая схема проверки не делает точно описывает модель, из поезда выигрывают (может быть оказан влияние из-за разделения набора данных), все еще хорошо отследить образцовую динамику улучшения - и таким образом как мы видим от этого примера, действительно хорошо прекратить повышать процесс ранее (прежде чем сверхустановка умрет),

# 3.3 Using Baseline Использование базовой линии

Возможно использовать предучебные результаты (основание) для обучения.

In [ ]:
current_params = params.copy()
current_params.update({
    'iterations': 10
})
model = CatBoostClassifier(**current_params).fit(X_train, y_train, categorical_features_indices)
# Get baseline (only with prediction_type='RawFormulaVal')
baseline = model.predict(X_train, prediction_type='RawFormulaVal')
# Fit new model
model.fit(X_train, y_train, categorical_features_indices, baseline=baseline);

# 3.4 Snapshot Support Поддержка снимков

Catboost поддерживает снимки. Вы можете использовать это для восстановления обучения после прерывания или для стартового обучения с предыдущими результатами.

In [ ]:
params_with_snapshot = params.copy()
params_with_snapshot.update({
    'iterations': 5,
    'learning_rate': 0.5,
    'logging_level': 'Verbose'
})
model = CatBoostClassifier(**params_with_snapshot).fit(train_pool, eval_set=validate_pool, save_snapshot=True)
params_with_snapshot.update({
    'iterations': 10,
    'learning_rate': 0.1,
})
model = CatBoostClassifier(**params_with_snapshot).fit(train_pool, eval_set=validate_pool, save_snapshot=True)

0:	learn: 0.8053892	test: 0.7937220	best: 0.7937220 (0)	total: 2.61ms	remaining: 10.4ms
1:	learn: 0.8008982	test: 0.7982063	best: 0.7982063 (1)	total: 8.55ms	remaining: 12.8ms
2:	learn: 0.8008982	test: 0.7937220	best: 0.7982063 (1)	total: 12.5ms	remaining: 8.34ms
3:	learn: 0.8113772	test: 0.7892377	best: 0.7982063 (1)	total: 18.6ms	remaining: 4.65ms
4:	learn: 0.8173653	test: 0.8026906	best: 0.8026906 (4)	total: 22.4ms	remaining: 0us

bestTest = 0.802690583
bestIteration = 4

5:	learn: 0.8173653	test: 0.8026906	best: 0.8026906 (4)	total: 24.8ms	remaining: 9.47ms
6:	learn: 0.8248503	test: 0.8026906	best: 0.8026906 (4)	total: 26.3ms	remaining: 5.92ms
7:	learn: 0.8233533	test: 0.8026906	best: 0.8026906 (4)	total: 28.4ms	remaining: 4ms
8:	learn: 0.8233533	test: 0.8026906	best: 0.8026906 (4)	total: 29.3ms	remaining: 1.72ms
9:	learn: 0.8233533	test: 0.8026906	best: 0.8026906 (4)	total: 31.3ms	remaining: 0us

bestTest = 0.802690583
bestIteration = 4



# 3.5 User Defined Objective Function Определенная пользователем  объективная функция

Возможно создать Вашу собственную объективную функцию. Давайте создадим logloss объективную функцию.

In [ ]:
class LoglossObjective(object):
    def calc_ders_range(self, approxes, targets, weights):
        # approxes, targets, weights are indexed containers of floats
        # (containers which have only __len__ and __getitem__ defined).
        # weights parameter can be None.
        #
        # To understand what these parameters mean, assume that there is
        # a subset of your dataset that is currently being processed.
        # approxes contains current predictions for this subset,
        # targets contains target values you provided with the dataset.
        #
        # This function should return a list of pairs (der1, der2), where
        # der1 is the first derivative of the loss function with respect
        # to the predicted value, and der2 is the second derivative.
        #
        # In our case, logloss is defined by the following formula:
        # target * log(sigmoid(approx)) + (1 - target) * (1 - sigmoid(approx))
        # where sigmoid(x) = 1 / (1 + e^(-x)).
        
        assert len(approxes) == len(targets)
        if weights is not None:
            assert len(weights) == len(approxes)
        result = []
        for index in range(len(targets)):
            e = np.exp(approxes[index])
            p = e / (1 + e)
            der1 = (1 - p) if targets[index] > 0.0 else -p
            der2 = -p * (1 - p)

            if weights is not None:
                der1 *= weights[index]
                der2 *= weights[index]

            result.append((der1, der2))
        return result    

In [ ]:
model = CatBoostClassifier(
    iterations=10,
    random_seed=42, 
    loss_function=LoglossObjective(), 
    eval_metric="Logloss"
)
# Fit model
model.fit(train_pool)
# Only prediction_type='RawFormulaVal' is allowed with custom `loss_function`
preds_raw = model.predict(X_test, prediction_type='RawFormulaVal')

0:	learn: 0.6827074	total: 384ms	remaining: 3.46s
1:	learn: 0.6723302	total: 392ms	remaining: 1.57s
2:	learn: 0.6619449	total: 394ms	remaining: 919ms
3:	learn: 0.6521466	total: 396ms	remaining: 594ms
4:	learn: 0.6435227	total: 399ms	remaining: 399ms
5:	learn: 0.6353848	total: 401ms	remaining: 267ms
6:	learn: 0.6277210	total: 404ms	remaining: 173ms
7:	learn: 0.6210282	total: 405ms	remaining: 101ms
8:	learn: 0.6141958	total: 408ms	remaining: 45.3ms
9:	learn: 0.6073236	total: 411ms	remaining: 0us


# 3.6 User Defined Metric Function Опаределпенная пользователеи функция

Также возможно создать Вашу собственную метрическую функцию. Давайте создадим logloss метрическую функцию.

In [ ]:
class LoglossMetric(object):
    def get_final_error(self, error, weight):
        return error / (weight + 1e-38)

    def is_max_optimal(self):
        return False

    def evaluate(self, approxes, target, weight):
        # approxes is a list of indexed containers
        # (containers with only __len__ and __getitem__ defined),
        # one container per approx dimension.
        # Each container contains floats.
        # weight is a one dimensional indexed container.
        # target is float.
        
        # weight parameter can be None.
        # Returns pair (error, weights sum)
        
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])

        approx = approxes[0]

        error_sum = 0.0
        weight_sum = 0.0

        for i in range(len(approx)):
            w = 1.0 if weight is None else weight[i]
            weight_sum += w
            error_sum += -w * (target[i] * approx[i] - np.log(1 + np.exp(approx[i])))

        return error_sum, weight_sum

In [ ]:
model = CatBoostClassifier(
    iterations=10,
    random_seed=42, 
    loss_function="Logloss",
    eval_metric=LoglossMetric()
)
# Fit model
model.fit(train_pool)
# Only prediction_type='RawFormulaVal' is allowed with custom `loss_function`
preds_raw = model.predict(X_test, prediction_type='RawFormulaVal')

Learning rate set to 0.5
0:	learn: 0.5521578	total: 217ms	remaining: 1.95s
1:	learn: 0.4885686	total: 220ms	remaining: 879ms
2:	learn: 0.4607664	total: 224ms	remaining: 524ms
3:	learn: 0.4418819	total: 226ms	remaining: 340ms
4:	learn: 0.4278162	total: 228ms	remaining: 228ms
5:	learn: 0.4151036	total: 230ms	remaining: 154ms
6:	learn: 0.4099336	total: 232ms	remaining: 99.5ms
7:	learn: 0.4095363	total: 233ms	remaining: 58.4ms
8:	learn: 0.4032867	total: 235ms	remaining: 26.2ms
9:	learn: 0.3929586	total: 237ms	remaining: 0us


# 3.7 Staged Predict Организованные предсказания

У модели CatBoost есть staged_predict метод. Это позволяет Вам многократно получать прогнозы для данного диапазона деревьев.

In [ ]:
model = CatBoostClassifier(iterations=10, random_seed=42, logging_level='Silent').fit(train_pool)
ntree_start, ntree_end, eval_period = 3, 9, 2
predictions_iterator = model.staged_predict(validate_pool, 'Probability', ntree_start, ntree_end, eval_period)
for preds, tree_count in zip(predictions_iterator, range(ntree_start, ntree_end, eval_period)):
    print('First class probabilities using the first {} trees: {}'.format(tree_count, preds[:5, 1]))

First class probabilities using the first 3 trees: [0.53597869 0.41039128 0.42057479 0.64281031 0.46576685]
First class probabilities using the first 5 trees: [0.63722688 0.42492029 0.46209302 0.70926021 0.44280772]
First class probabilities using the first 7 trees: [0.66964764 0.42409144 0.46124982 0.76101033 0.47205986]


# 3.8 Feature Importances

Иногда очень важно понять, какая особенность сделала самый большой вклад в конечный результат. Чтобы сделать это, у модели CatBoost есть get_feature_importance метод.

In [ ]:
model = CatBoostClassifier(iterations=50, random_seed=42, logging_level='Silent').fit(train_pool)
feature_importances = model.get_feature_importance(train_pool)
feature_names = X_train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

Sex: 59.004092014268586
Pclass: 16.340887169747035
Ticket: 6.028107169932204
Cabin: 3.8347242202560192
Fare: 3.712969667934384
Age: 3.484451204182482
Parch: 3.378089740355865
Embarked: 2.3139994072899555
SibSp: 1.9026794060334504
PassengerId: 0.0
Name: 0.0


Это показывает, что показывает Sex, и Pclass имел самое большое влияние на результат.

# 3.9 Eval Metrics

У CatBoost есть eval_metrics метод, который позволяет вычислять данный метрики на данный набор данных. И потянуть их, конечно:)

In [ ]:
model = CatBoostClassifier(iterations=50, random_seed=42, logging_level='Silent').fit(train_pool)
eval_metrics = model.eval_metrics(validate_pool, ['AUC'], plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [ ]:
print(eval_metrics['AUC'][:6])

[0.8627368774106994, 0.8623176253563642, 0.8602213650846889, 0.8514170719436525, 0.8495723629045783, 0.8569092738554419]


# 3.10 Learning Processes Comparison

Вы можете также сравнить различный процесс обучения моделей на единственном участке.

In [ ]:
model1 = CatBoostClassifier(iterations=10, depth=1, train_dir='model_depth_1/', logging_level='Silent')
model1.fit(train_pool, eval_set=validate_pool)
model2 = CatBoostClassifier(iterations=10, depth=5, train_dir='model_depth_5/', logging_level='Silent')
model2.fit(train_pool, eval_set=validate_pool);

In [ ]:
from catboost import MetricVisualizer
widget = MetricVisualizer(['model_depth_1', 'model_depth_5'])
widget.start()

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

# 3.11 Model Saving

Всегда действительно удобно быть в состоянии сохранить Вашу модель на диске (особенно, если обучение заняло время).

In [ ]:
model = CatBoostClassifier(iterations=10, random_seed=42, logging_level='Silent').fit(train_pool)
model.save_model('catboost_model.dump')
model = CatBoostClassifier()
model.load_model('catboost_model.dump');

# 4 Parameters Tuning

В то время как Вы могли всегда выбирать оптимальное количество повторений (повышающий шаги) перекрестной проверкой и сюжетами кривой обучения, также важно играть с некоторыми образцовыми параметрами, и мы хотели бы обратить некоторое особое внимание на l2_leaf_reg и learning_rate.

В этом разделе мы выберем эти параметры, используя гипервыбирать пакет.

In [ ]:
import hyperopt

def hyperopt_objective(params):
    model = CatBoostClassifier(
        l2_leaf_reg=int(params['l2_leaf_reg']),
        learning_rate=params['learning_rate'],
        iterations=500,
        eval_metric='Accuracy',
        random_seed=42,
        verbose=False,
        loss_function='Logloss',
    )
    
    cv_data = cv(
        Pool(X, y, cat_features=categorical_features_indices),
        model.get_params()
    )
    best_accuracy = np.max(cv_data['test-Accuracy-mean'])
    
    return 1 - best_accuracy # as hyperopt minimises

In [ ]:
from numpy.random import RandomState

params_space = {
    'l2_leaf_reg': hyperopt.hp.qloguniform('l2_leaf_reg', 0, 2, 1),
    'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 5e-1),
}

trials = hyperopt.Trials()

best = hyperopt.fmin(
    hyperopt_objective,
    space=params_space,
    algo=hyperopt.tpe.suggest,
    max_evals=50,
    trials=trials,
    rstate=RandomState(123)
)

print(best)

100%|██████████| 50/50 [07:52<00:00,  9.46s/it, best loss: 0.1661054994388328]
{'l2_leaf_reg': 1.0, 'learning_rate': 0.030276027601240763}


Теперь давайте получим все данные об условной цене с лучшими параметрами:

In [ ]:
model = CatBoostClassifier(
    l2_leaf_reg=int(best['l2_leaf_reg']),
    learning_rate=best['learning_rate'],
    iterations=500,
    eval_metric='Accuracy',
    random_seed=42,
    verbose=False,
    loss_function='Logloss',
)
cv_data = cv(Pool(X, y, cat_features=categorical_features_indices), model.get_params())

In [ ]:
print('Precise validation accuracy score: {}'.format(np.max(cv_data['test-Accuracy-mean'])))

Precise validation accuracy score: 0.8338945005611672


Вспомните, что с параметрами по умолчанию счет условной цены был 0.8283, и таким образом у нас есть (вероятно, не статистически значительный) некоторое улучшение.

# Make submission Сделайте представление

Теперь мы переобучили бы нашу настроенную модель на всех данных о train, которые мы имеем

In [ ]:
model.fit(X, y, cat_features=categorical_features_indices)

И наконец давайте подготовим файл представления:

In [ ]:
import pandas as pd
submisstion = pd.DataFrame()
submisstion['PassengerId'] = X_test['PassengerId']
submisstion['Survived'] = model.predict(X_test)

In [ ]:
submisstion.to_csv('submission.csv', index=False)

Наконец Вы можете сделать представление на колоссальном соревновании Kaggle.

Вот именно! Теперь Вы можете играть вокруг с CatBoost и выиграть некоторые соревнования!:)